In [1]:
import argparse

In [3]:
help(argparse.ArgumentParser)

Help on class ArgumentParser in module argparse:

class ArgumentParser(_AttributeHolder, _ActionsContainer)
 |  ArgumentParser(prog=None, usage=None, description=None, epilog=None, parents=[], formatter_class=<class 'argparse.HelpFormatter'>, prefix_chars='-', fromfile_prefix_chars=None, argument_default=None, conflict_handler='error', add_help=True, allow_abbrev=True, exit_on_error=True)
 |  
 |  Object for parsing command line strings into Python objects.
 |  
 |  Keyword Arguments:
 |      - prog -- The name of the program (default:
 |          ``os.path.basename(sys.argv[0])``)
 |      - usage -- A usage message (default: auto-generated from arguments)
 |      - description -- A description of what the program does
 |      - epilog -- Text following the argument descriptions
 |      - parents -- Parsers whose arguments should be copied into this one
 |      - formatter_class -- HelpFormatter class for printing help messages
 |      - prefix_chars -- Characters that prefix optional 

In [4]:
from chemicalx.data import DrugPairBatch, BatchGenerator, DrugCombDB

loader = DrugCombDB()



context_set = loader.get_context_features()
drug_set = loader.get_drug_features()
triplets = loader.get_labeled_triples()


generator = BatchGenerator(batch_size=32,context_features=True, drug_features=True,drug_molecules=True,
                           context_feature_set=context_set, drug_feature_set=drug_set, labeled_triples=train)

NameError: name 'train' is not defined

TypeError: object of type 'DrugCombDB' has no len()

In [6]:
len(triplets)

191391

In [7]:
len(drug_set)

2956

In [21]:

from  chemicalx.data import BatchGenerator, DrugComb, DrugCombDB, OncoPolyPharmacology
from chemicalx.data import DrugPairBatch

import pandas as pd

'''
    Stratified KFold should be subclassed from `chemicalx` package
'''


class DataLoader(BatchGenerator):
    
    def __init__(self):
        # self.model = args.model
        self.dataset = "drugcomb"
        # self.compute_embeddings = args.compute_embeddings
        # self.validation = args.validation
        drug_set, context_set, train, self.test = self.get_datasets()
        super(DataLoader,self).__init__(batch_size=32,context_features=True, drug_features=True,drug_molecules=True,
                           context_feature_set=context_set, drug_feature_set=drug_set, labeled_triples=train)
        
    
    def get_datasets(self):
        if self.dataset == "drugcomb":
            loader = DrugComb()
        elif self.dataset == "drucombdb":
            loader = DrugCombDB()
        elif self.dataset == "oncolypharma":
            loader = OncoPolyPharmacology()
        else:
            print(f"The given dataset {self.dataset} is not a part of chemicalx, provide the 3 synergy prediction tasks")

        drug_set = loader.get_drug_features()
        context_set = loader.get_context_features()
        triplets = loader.get_labeled_triples()
        train, test = triplets.train_test_split(train_size=0.8)
        return drug_set, context_set, train, test
    

    def get_testloader(self):
        self.labeled_triples = self.test
        




In [23]:
x = DataLoader()

In [24]:
for i in x:
    sample = i
print(sample)

DrugPairBatch(identifiers=          drug_1    drug_2     context  label
532891  24776445  54676905  LB2518-MEL    0.0
54546     132971      5790     HS 578T    0.0
551638  16124726   9943465     RVH-421    1.0
306501    104842    163659    NCI-H522    1.0
622179  11977753  25022668     CBRC008    0.0
24815   54611422      3685      SNB-19    1.0
405789   5329102      5799         M14    1.0
491679  11640390    644213       A101D    0.0
501594   3463933   5278396    COLO 800    0.0
279629  24958200   3062316    NCIH1650    1.0, drug_features_left=tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]]), drug_molecules_left=PackedGraph(batch_size=10, num_nodes=[27, 26, 32, 29, 36, 34, 29, 32, 22, 24], num_edges=[58, 60, 72, 66, 82, 72, 62, 74, 46, 54], num_relation=4), drug_features_right=tenso

In [2]:
import pandas as pd
pd.read_csv("../datasets/embeddings_drugcombdb.csv")

,Drugs,Drug Embeddings,mrr,hits@10
0,['65628'],"[-9.71540355682373, -11.125833511352539, -7.97...",0.363178,0.854167
1,['39562'],"[-9.794931411743164, -8.03378963470459, -12.02...",0.292606,0.706522
2,['216326'],"[-8.874168395996094, -4.353018760681152, 2.002...",0.417559,0.988095
3,['20279'],"[0.9404932856559753, -6.391180515289307, -0.40...",0.416144,0.964286
4,['439693'],"[1.144148349761963, -6.067200183868408, 0.0682...",0.362388,0.964286
...,...,...,...,...
2951,['24197464'],"[1.0625643730163574, -6.0400519371032715, -0.3...",0.368407,0.961538
2952,['54611422'],"[2.236278772354126, -5.427398681640625, 1.5881...",0.345175,0.979167
2953,['441203'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000
2954,['54611489'],"[3.107060432434082, -5.344871520996094, 0.7818...",0.337139,0.988281


In [12]:
import torch
import torchdrug.data
from torch.types import Device
from src.utils import plot_embeddings

__all__ = [
    "PackedGraph",
    "Graph",
]

class PackedGraph(torchdrug.data.PackedGraph):
    def to(self, device: Device):
        if isinstance(device, str):
            if device == "cpu":
                return self.cpu()
            elif device == "cuda":
                return self.cuda()
            else:
                raise NotImplementedError(f"{self.__class__.__name__}.to() is not implemented for string: {device}")
        elif isinstance(device, torch.device):
            if device.type == "cpu":
                return self.cpu()
            elif device.type == "cuda":
                return self.cuda()
            else:
                raise NotImplementedError
        else:
            raise TypeError


class Graph(torchdrug.data.Graph):
    packed_type = PackedGraph



from typing import List, Optional

import torch
from torch import nn
from torch.nn.functional import normalize
from torchdrug.layers import MaxReadout
from torchdrug.layers import MultiLayerPerceptron as MLP
from torchdrug.models import GraphConvolutionalNetwork, GraphAttentionNetwork, GraphIsomorphismNetwork

# from compat import PackedGraph
from chemicalx.constants import TORCHDRUG_NODE_FEATURES
from chemicalx.data import DrugPairBatch
from chemicalx.models import Model

In [5]:
from chemicalx.data import DrugPairBatch, BatchGenerator, DrugCombDB

loader = DrugCombDB()



context_set = loader.get_context_features()
drug_set = loader.get_drug_features()
triplets = loader.get_labeled_triples()

train, test = triplets.train_test_split(train_size=0.8)

loader = BatchGenerator(batch_size=32,context_features=True, drug_features=True,drug_molecules=True,
                           context_feature_set=context_set, drug_feature_set=drug_set, labeled_triples=train)

In [9]:
len(loader.context_feature_set)

112

In [86]:
sample.drug_features_right

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 2.],
        [0., 5., 0.,  ..., 4., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 1.],
        [0., 2., 0.,  ..., 0., 0., 0.]])

In [74]:
for i in sample.drug_molecules_left:
    sample1 = i

In [83]:
sample1

KeyError: '__name__'

In [81]:
help(sample1.data_dict)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Built-in subclasses:
 |      StgDict
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>va

In [67]:
train.data

,drug_1,drug_2,context,label
147626,123631,457193,ACHN,1.0
40359,447043,115355,KBM-7,1.0
74212,36314,123608,HT29,0.0
185454,54600319,32874,RPMI-8226,1.0
57367,667490,6445540,LOX IMVI,0.0
...,...,...,...,...
119879,2708,387447,MCF7,0.0
103694,4114,545,IGROV1,0.0
131932,25126798,3367,CCRF-CEM,0.0
146867,25136944,457193,SK-MEL-2,1.0


In [60]:
x = 0
for i in loader:
    x = x+1
print(x)

1


In [ ]:
from transformers import AutoModel
from ultra.datasets import HM
import torch
from torch_geometric.data import Data
from ultra.tasks import build_relation_graph
from collections import UserDict
from typing import Dict, Iterable, Mapping, Union
from chemicalx.data import DrugCombDB, BatchGenerator, DrugComb, OncoPolyPharmacology
import torch
from torchdrug.data import Molecule,PackedGraph,Graph
import math
import torch
from torch import distributed as dist
from torch.utils import data as torch_data
from torch_geometric.data import Data
from ultra import tasks, util
from torch_geometric.data import Data, InMemoryDataset, download_url, extract_zip
import os
from ultra.tasks import build_relation_graph
import pandas as pd
from molecule import MoleculesData
import json
from tqdm import tqdm
import utils as mu
import numpy as np 
import gc
import psutil



model = AutoModel.from_pretrained("mgalkin/ultra_3g", trust_remote_code=True)

loader = DrugCombDB()

context_set = loader.get_context_features()
drug_set = loader.get_drug_features()
triplets = loader.get_labeled_triples()

train, test = triplets.train_test_split(train_size=0.8)


def log_memory_usage(step):
    process = psutil.Process()
    memory_info = process.memory_info()
    # print(f"[{step}] Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")

                           
def all_negative(data, batch):
    pos_h_index, pos_t_index, pos_r_index = batch
    r_index = pos_r_index.unsqueeze(-1).expand(-1, data.num_nodes)
    # generate all negative tails for this batch
    all_index = torch.arange(data.num_nodes)
    h_index, t_index = torch.meshgrid(pos_h_index, all_index, indexing="ij")  # indexing "xy" would return transposed
    t_batch = torch.stack([h_index, t_index, r_index], dim=-1)
    # generate all negative heads for this batch
    all_index = torch.arange(data.num_nodes)
    t_index, h_index = torch.meshgrid(pos_t_index, all_index, indexing="ij")
    h_batch = torch.stack([h_index, t_index, r_index], dim=-1)

    return t_batch, h_batch




def save_edge_lists(edge_lists_save,i):
    with open(f"datasets_edges/test{str(i)}.txt", 'w') as file:
        for edge in edge_lists_save:
            file.write('\t'.join(map(str, edge)) + '\n')

def save_edges(edge_lists_save):
    with open(f"test.txt", 'w') as file:
        for edge in edge_lists_save:
            file.write('\t'.join(map(str, edge)) + '\n')



drug_embeddings = []
drugs = []
mrr = []
hits10 = []
for ind,i in tqdm(enumerate(drug_set)):
    edge_lists = []
    x = Graph.pack([drug_set[i]["molecule"]])

    for j in x.edge_list.cpu().numpy().tolist():
        x1 = [j[0],j[2],j[1]]
        edge_lists.append(j)
    

    # save_edge_lists(edge_lists,ind)  
    # save_edges(edge_lists)
    embeddings  = []
  
    md = MoleculesData()
    data = md.process()
    device = util.get_devices(None)
    world_size = util.get_world_size()
    rank = util.get_rank()
    sampler = torch_data.DistributedSampler(edge_lists, world_size, rank)
    test_loader = torch_data.DataLoader(edge_lists, len(edge_lists), sampler=sampler)    
    model.eval()
    
    try:
        res = mu.test(model,data,edge_lists)
        mrr.append(res[0].cpu().item())
        hits10.append(res[1].cpu().item())

    except:
        mrr.append(0)
        hits10.append(0)
        print(f"Wrong in Dataloader")
    
    for batch in test_loader:
        t_batch, h_batch = all_negative(data, batch)
        try:
            t_pred,node_feature = model(data, t_batch)
            h_pred,_ = model(data, h_batch)
            emb = h_pred.reshape(-1).detach().numpy()
        except:
            print(f"Some Error in model - Check once file no {ind} and drug {i}")
            t_pred = np.zeros((2,3))
            h_pred = np.zeros((2,3))
            drug_df = pd.concat([pd.DataFrame(t_pred),pd.DataFrame(h_pred)],axis=1)
            emb = np.array(drug_df).reshape(-1)
        embeddings.append(emb)
    embeddings = np.array(embeddings).reshape(-1).tolist()
    drug_embeddings.append(embeddings)
    drugs.append([i])
    log_memory_usage(f"After processing drug {ind}")

    # Clear unnecessary variables and collect garbage
    del edge_lists, embeddings, data, test_loader, sampler
    torch.cuda.empty_cache()
    gc.collect()




print("----1st for ")
def pad(drug_embeddings):
    padding = max(len(embedding) for embedding in drug_embeddings)
    padded_embeddings = []
    for embedding in drug_embeddings:
        padded_embedding = np.pad(embedding, (0, padding - len(embedding)), 'constant')
        padded_embeddings.append(padded_embedding.tolist()) 
        del padded_embedding
    return padded_embeddings



padded_embeddings =  pad(drug_embeddings)
print("----2nd for ")
padding = max(len(embedding) for embedding in drug_embeddings)
embedding_drug_set = {}
for i in range(len(drugs)):
    embedding_drug_set[drugs[i][0]] = padded_embeddings[i]


with open('embedding_drug_set.json', 'w') as json_file:
    json.dump(embedding_drug_set, json_file)



print("----Saving ")
benchmark = pd.DataFrame([drugs,padded_embeddings,mrr,hits10,[padding]]).T
benchmark.columns = ["Drugs", "Drug Embeddings","mrr","hits@10","padding"]
benchmark.to_csv("results.csv",index=False)

drug_embeddings_str = [json.dumps(embedding) for embedding in padded_embeddings]

benchmark = pd.DataFrame({'Drugs': drugs, 'Drug Embeddings': drug_embeddings_str, 'mrr': mrr, 'hits@10': hits10})
benchmark.to_csv("results1.csv", index=False)



In [14]:
from torch_geometric.data import Dataset

In [18]:
def sort_runs(run_strings):
    # Extract numerical part, convert to int, and sort
    numbers = sorted(int(run[4:]) for run in run_strings)
    return numbers

# Example usage
run_strings = ["runs3", "runs1", "runs2"]
sorted_numbers = sort_runs(run_strings)
print(sorted_numbers[-1]+1)


4


In [22]:
import pandas as pd
from sklearn.metrics import classification_report

class TrainingMetrics:
    def get_training_metrics(self, loss, report):
        """
        Parses the training report and loss to generate a linear pandas DataFrame with training metrics.

        Parameters:
        - loss: The training loss (assumed to be a float or a list of loss values).
        - report: The training report containing metrics (assumed to be a dictionary).

        Returns:
        - A pandas DataFrame with the parsed training metrics.
        """
        # Step 1: Parse the loss (assuming it's a single value or a list of values)
        if isinstance(loss, list):
            avg_loss = sum(loss) / len(loss)
        else:
            avg_loss = loss

        # Step 2: Parse the report (assuming it's a dictionary)
        if isinstance(report, dict):
            metrics = report
        else:
            raise ValueError("Report should be a dictionary")

        # Step 3: Convert metrics to a pandas DataFrame
        df = pd.DataFrame(metrics).transpose()



        # Step 5: Flatten the DataFrame
        df_flat = df.unstack().to_frame().T
        df_flat.columns = [f'{col[0]}_{col[1]}' for col in df_flat.columns]

        return df_flat

# Example usage
y_true = [0, 1, 1, 0]
y_pred = [0, 0, 1, 1]

loss = 0.5
report_dict = classification_report(y_true, y_pred, output_dict=True)

tm = TrainingMetrics()
metrics_df = tm.get_training_metrics(loss, report_dict)
print(metrics_df)
metrics_df


   precision_0  precision_1  precision_accuracy  precision_macro avg  \
0          0.5          0.5                 0.5                  0.5   

   precision_weighted avg  recall_0  recall_1  recall_accuracy  \
0                     0.5       0.5       0.5              0.5   

   recall_macro avg  recall_weighted avg  f1-score_0  f1-score_1  \
0               0.5                  0.5         0.5         0.5   

   f1-score_accuracy  f1-score_macro avg  f1-score_weighted avg  support_0  \
0                0.5                 0.5                    0.5        2.0   

   support_1  support_accuracy  support_macro avg  support_weighted avg  
0        2.0               0.5                4.0                   4.0  


,precision_0,precision_1,precision_accuracy,precision_macro avg,precision_weighted avg,recall_0,recall_1,recall_accuracy,recall_macro avg,recall_weighted avg,f1-score_0,f1-score_1,f1-score_accuracy,f1-score_macro avg,f1-score_weighted avg,support_0,support_1,support_accuracy,support_macro avg,support_weighted avg
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,2.0,2.0,0.5,4.0,4.0


In [15]:
help(Dataset)

Help on class Dataset in module torch_geometric.data.dataset:

class Dataset(torch.utils.data.dataset.Dataset)
 |  Dataset(root: Optional[str] = None, transform: Optional[Callable] = None, pre_transform: Optional[Callable] = None, pre_filter: Optional[Callable] = None, log: bool = True, force_reload: bool = False) -> None
 |  
 |  Dataset base class for creating graph datasets.
 |  See `here <https://pytorch-geometric.readthedocs.io/en/latest/tutorial/
 |  create_dataset.html>`__ for the accompanying tutorial.
 |  
 |  Args:
 |      root (str, optional): Root directory where the dataset should be saved.
 |          (optional: :obj:`None`)
 |      transform (callable, optional): A function/transform that takes in a
 |          :class:`~torch_geometric.data.Data` or
 |          :class:`~torch_geometric.data.HeteroData` object and returns a
 |          transformed version.
 |          The data object will be transformed before every access.
 |          (default: :obj:`None`)
 |      pre_t

In [15]:
for i in loader:
    sample = i

In [16]:
sample

DrugPairBatch(identifiers=           drug_1    drug_2     context  label
45054     3034010     60838       KBM-7    1.0
88400        2123      2141      HOP-62    0.0
172321   24776445  11525740    NCI-H226    1.0
82025      248862  24197464        EKVX    0.0
32015        8226  54680692       KBM-7    0.0
158504   42611257      3950      SF-295    1.0
81496       77082    123608      MOLT-4    0.0
33571     5284592      5426       KBM-7    0.0
47627       41684      3019       KBM-7    0.0
67608    54608520      2907       K-562    0.0
138921    3081361  25126798   SK-MEL-28    1.0
187840    5352062     32874      OVCAR3    0.0
142414  135398738    457193     OVCAR-5    1.0
82701     2733525    248862      SW-620    0.0
18204        5394  46239015      SW-620    0.0
111892     356653      5426      HOP-92    0.0
179063   25126798    216326    CCRF-CEM    0.0
151778      71384      2578       UO-31    0.0
98597        2187      5746      OVCAR3    0.0
131424     123631    208908    CCR

In [56]:

import torch
import torch.nn as nn



class UltraNet(nn.Module):

    def __init__(self,input_dims):
        super().__init__()
        self.input_dim = input_dims
        self.lin1 = nn.Linear(in_features=self.input_dim,out_features=32)

    

    def get_embeddings(self,molecules):
        drug_embedding_batch = []
        for i in molecules:
            edge_lists = []
            for j in i.edge_list.cpu().numpy().tolist():
                x1 = [j[0],j[2],j[1]]
                edge_lists.append(j)
            
            embeddings  = []
  
            md = MoleculesData()
            data = md.process()
            device = util.get_devices(None)
            world_size = util.get_world_size()
            rank = util.get_rank()
            sampler = torch_data.DistributedSampler(edge_lists, world_size, rank)
            test_loader = torch_data.DataLoader(edge_lists, len(edge_lists), sampler=sampler)    
            model.eval()
            for batch in test_loader:
                t_batch, h_batch = all_negative(data, batch)
                try:
                    t_pred,node_feature = model(data, t_batch)
                    h_pred,_ = model(data, h_batch)
                    emb = h_pred.reshape(-1).detach().numpy()
                except:
                    print(f"Some Error in model - Check once file no {ind} and drug {i}")
                    t_pred = np.zeros((2,3))
                    h_pred = np.zeros((2,3))
                    drug_df = pd.concat([pd.DataFrame(t_pred),pd.DataFrame(h_pred)],axis=1)
                    emb = np.array(drug_df).reshape(-1)
                embeddings.append(emb)
            drug_embedding_batch.append(embeddings)
        return torch.tensor(drug_embedding_batch,dtype=torch.float32)
        


    def forward(self,  molecules_left: PackedGraph, molecules_right: PackedGraph, plot=False,
    ) -> torch.FloatTensor:
            
        drug_embedding_left = self.get_embeddings(molecules_left)
        x = self.lin1(drug_embedding_left)

        return drug_embedding_left


    

In [53]:
model = UltraNet(input_dims=48980)

In [46]:
sample.drug_molecules_left

Graph(num_node=35, num_edge=70, num_relation=4)

In [40]:
sample.identifiers

,drug_1,drug_2,context,label
45054,3034010,60838,KBM-7,1.0
88400,2123,2141,HOP-62,0.0
172321,24776445,11525740,NCI-H226,1.0
82025,248862,24197464,EKVX,0.0
32015,8226,54680692,KBM-7,0.0
158504,42611257,3950,SF-295,1.0
81496,77082,123608,MOLT-4,0.0
33571,5284592,5426,KBM-7,0.0
47627,41684,3019,KBM-7,0.0
67608,54608520,2907,K-562,0.0


In [55]:
model(sample.drug_molecules_left, sample.drug_molecules_right)

NameError: name 'MoleculesData' is not defined

In [22]:
sample

DrugPairBatch(identifiers=           drug_1    drug_2     context  label
45054     3034010     60838       KBM-7    1.0
88400        2123      2141      HOP-62    0.0
172321   24776445  11525740    NCI-H226    1.0
82025      248862  24197464        EKVX    0.0
32015        8226  54680692       KBM-7    0.0
158504   42611257      3950      SF-295    1.0
81496       77082    123608      MOLT-4    0.0
33571     5284592      5426       KBM-7    0.0
47627       41684      3019       KBM-7    0.0
67608    54608520      2907       K-562    0.0
138921    3081361  25126798   SK-MEL-28    1.0
187840    5352062     32874      OVCAR3    0.0
142414  135398738    457193     OVCAR-5    1.0
82701     2733525    248862      SW-620    0.0
18204        5394  46239015      SW-620    0.0
111892     356653      5426      HOP-92    0.0
179063   25126798    216326    CCRF-CEM    0.0
151778      71384      2578       UO-31    0.0
98597        2187      5746      OVCAR3    0.0
131424     123631    208908    CCR